## Regularized logistic regression

The new cost function $J(\theta)$ includes a squared penalty with regularization parameter $\lambda$:

$J(\theta) = - (m^{-1} \sum_{i = 1}^{m} y_i log h_{\theta} (x_i) + (1 - y_i) log (1 - h_{\theta} (x_i))) + \frac{\lambda}{2m} \sum_{j =1} ^ n \theta_{j}^{2}$  

When $\lambda$ is large, more regularization occurs.

In [141]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.special import expit
from scipy.optimize import minimize
from __future__ import division
from math import log
plt.style.use('ggplot')

def sim_reg(N, M):
    ''' simulates logistic regression data '''
    intercept = np.repeat(1, N)
    if M > 1:
        x = np.random.rand(N, M - 1)
        X = np.column_stack((intercept, x))
    else:
        X = intercept
    beta = np.random.normal(0, 1, M)
    h_theta = expit(np.dot(X, beta))
    y = np.random.binomial(1, h_theta, N)
    return {'beta':beta, 'y':y, 'X':X}

out = sim_reg(N = 1000, M = 5)

Next I'll define some helper functions to compute the log likelihood and the cost function (negative log likelihood + regularization term).

In [142]:
def loglik(theta, y, x):
    ''' computes logistic regression log likelihood '''
    p = expit(np.dot(x, theta))
    res = y * log(p) + (1 - y) * log(1 - p)
    return res

def cost_fun(theta, lam, y, X):
    ''' computes cost function '''
    
    M = len(X[0, :]) # number of parameters
    N = len(X[:, 0]) # number of observations
    
    # cost component due to regularization
    reg_cost = lam / (2.0 * N) * sum(theta[1:] ** 2)
    
    # cost due to lack of fit
    fit_cost = [loglik(theta, y[i], X[i, :]) for i in range(N)]
    fit_cost = - sum(fit_cost) / M
    
    cost = fit_cost + reg_cost
    return cost

Then I'll come up with some initial values $\theta_{\text{init}}$ and optimize to find the theta that minimize the cost function.

In [148]:
theta_init = np.random.normal(0, 0.1, len(out['X'][0, :]))

res = minimize(cost_fun, theta_init, 
               method='nelder-mead', 
               options={'xtol': 1e-8},
               args=(0, out['y'], out['X']))

print theta_init
print res['x']
print out['beta']

[ 0.09639041 -0.05206089  0.01227324  0.11676205 -0.17258369]
[ 1.6311084  -0.54261767 -0.82622108  0.0322602  -1.66068865]
[ 1.33024274 -0.26130636 -0.66052471 -0.0857613  -1.41851758]
